Import the library functionality

In [1]:
from BoringTUN import *

## Keys and key pairs

Generate brand new keypair

In [2]:
p1 = KeyPair(sec=None, pub=None)
p1

KeyPair(sec=SecretKey<!!!SECRET_DATA!!!>, pub=PublicKey(x25519_key.fromBytes(b'>"\xc6\xee\x10>\x90\x89\xb4\x174\xf7\xbd\x92\xfb\x95-3\x8a\xa5\xe0\xd1\xe8\x91h\x98Z\x98b~,s')))

Derive public key from private one

In [3]:
p1 = KeyPair(sec=p1.sec, pub=None)
p1

KeyPair(sec=SecretKey<!!!SECRET_DATA!!!>, pub=PublicKey(x25519_key.fromBytes(b'>"\xc6\xee\x10>\x90\x89\xb4\x174\xf7\xbd\x92\xfb\x95-3\x8a\xa5\xe0\xd1\xe8\x91h\x98Z\x98b~,s')))

## Configs

In [4]:
from BoringTUN.config import *

Configs correspond to `wg-quick` config files.

In [5]:
p2 = KeyPair(sec=None, pub=None)
cfg1 = WGConfig(
    interface=Interface(p1.sec),
    peers=[
        Peer(
            pub=p2.pub,
            ip=None, # needed for real, non-simulated connections
            port=None,
            psk="AgGZWT8Gp2la+dkmDWPxMVTp1WJgR4gmAubGu9Z6crg=",
            keepAliveTimeout=10,
        )
    ]
)

While we could have used the same keypair, let's generate a new one.

In [6]:
cfg2 = WGConfig(
    interface=Interface(p2.sec),
    peers=[
        Peer(
            pub=p1.pub,
            ip=None, # needed for real, non-simulated connections
            port=None,
            psk=cfg1.peers[0].psk,
            keepAliveTimeout=10,
        )
    ]
)

## Tunnels
Let's create new tunnel. They are context managers.

In [7]:
t1 = Tunnel(cfg1.interface, cfg1.peers[0]).__enter__()
t2 = Tunnel.fromConfig(cfg2).__enter__()

Let's do a handshake

In [8]:
hs = t1.force_handshake()
hs

Action(<Opcode.WRITE_TO_NETWORK: 1>, b'\x01\x00\x00\x00\x01\x00\x00\x00\xe5\x8aC\xb4h\xbaC\x14\x1fA\xbeZ\t\xe6\x9f\x8f\xe6\x9b\x92V\x91\n\xd1\xfa\xb5"\tH\xd1VwU\'s%\x81\x1a\x9e\x81=D\xc8\xa8\x1eY\xcd8\xf4\xf9X\x82n\xd6\xed\x02\xf3f(\xaf\x99\xe5\xe0sb\xe6\xd9\xa883\xb8\xea\xd2\xd9[\x10\x10p\xaa\xa1L\x13\x15O)t\xb4Q\xb0\xb6\x0f\x16O\xb8|\xec\xb5\xc7)dboDV\xa0\xe1YSO\xf1\x8c"\xbc\x1aJMR\x0f\xcfm\xb4\x95S\xe5\xa3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [9]:
hs = t2.unwrap(hs.buf)
hs

Action(<Opcode.WRITE_TO_NETWORK: 1>, b'\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x97\xd4^B\xe5_)\xb0I\x9e\x85z\xc4\xd3\x0c\x9dkry\xcc\xfdj\x9a\x84\xab\xf4\xf9\x1d\xbf?\x118\xc6;\x9b{\xab\xc7\xd0\xa7\xc7\xa3\n\x84\xc94\xf5\x81\xee\xc3\xac\x82\x1d\xb1\x08\x14\x12\xd2v\xe1b)|r\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [10]:
hs = t1.unwrap(hs.buf)
hs

Action(<Opcode.WRITE_TO_NETWORK: 1>, b'\x04\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00o\xa7\xaf~5Z\x15\xd3\xba\x9a\xbc\xa5it\x98P')

In [11]:
hs = t2.unwrap(hs.buf)
hs

Action(<Opcode.WIREGUARD_DONE: 0>)

Now we can transfer data

In [12]:
pingPacket = (
	b"\x45\x00\x00\x54\x84\xcb\x40\x00\x40\x01\xb7\xdb\x7f\x00\x00\x01\x7f\x00\x00\x01"  # IPv4 header
	+	b"\x08\x00\x19\xc2\x00\x0e\x00\x01\x41\x3d\x5d\x62\x00\x00\x00\x00\x7b\xbc\x05\x00\x00\x00\x00\x00\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f\x20\x21\x22\x23\x24\x25\x26\x27\x28\x29\x2a\x2b\x2c\x2d\x2e\x2f\x30\x31\x32\x33\x34\x35\x36\x37"  # ICMP packet
)

In [13]:
tx = t1.wrap(pingPacket)
tx

Action(<Opcode.WRITE_TO_NETWORK: 1>, b'\x04\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\xd7\xf0 8C\xe6\x80@\xe3e\x07f;\xcci\xdf#v\xc1\r\x14\xa0\x9b\xdb\xdb\xf2\xe3\xf7\x99\xa4\x91\xcf\xab\xb7\x9fr\xc5;`=\x10\xa4A\x812\xb0\x8a\xb4\xe131:\xe5!\x81\xf1\xecs\x92y\x982$\xe7e\x1b\x0b\x88Rr:qlT\x1f\x8c\xdb\xe4\x82\x00v\xaa\xe8\xa8\xd0|\xca\xd5\xff\x95\xbdJ\xec\xb1\\\x86\x99\xachn')

In [14]:
rx = t2.unwrap(tx.buf)
rx

Action(<Opcode.WRITE_TO_TUNNEL_IPV4: 4>, b'E\x00\x00T\x84\xcb@\x00@\x01\xb7\xdb\x7f\x00\x00\x01\x7f\x00\x00\x01\x08\x00\x19\xc2\x00\x0e\x00\x01A=]b\x00\x00\x00\x00{\xbc\x05\x00\x00\x00\x00\x00\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567')

In [15]:
rx.buf == pingPacket

True

As you see, the type of the packet is detected automatically by BoringTUN. But piping arbitrary data doesn't work:

In [16]:
tx = t1.wrap(b"abcdefghijklmnopywrtuvwxyz")
rx = t2.unwrap(tx.buf)
rx

Action(<Opcode.WIREGUARD_ERROR: 2>, b'abcdefghijk')

One should pass ticks by timer

In [17]:
tx = t1.tick()
tx

Action(<Opcode.WIREGUARD_DONE: 0>)

One can get internal stats:

In [18]:
s1 = t1.stats()
s2 = t2.stats()
display(s1, s2)

stats<time_since_last_handshake=1653667872, tx_bytes=110, rx_bytes=0, estimated_loss=0.0, estimated_rtt=199, reserved=<BoringTUN.ctypes.c_ubyte_Array_56 object at 0x7f7ee0697240>>

stats<time_since_last_handshake=1653667872, tx_bytes=0, rx_bytes=84, estimated_loss=0.0, estimated_rtt=-1, reserved=<BoringTUN.ctypes.c_ubyte_Array_56 object at 0x7f7ee0697240>>

## `asyncio`

Module `BoringTUN.asyncio` contains `BoringTUNProtocol` class, which implements [both a `Transport` and a `DatagramProtocol`](https://docs.python.org/3/library/asyncio-protocol.html). Though it gets and consumes raw IP packets, not UDP ones. It's up to you to parse and process them.